# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [11]:
import graphlab
import numpy as np

# convenient functions
squared = lambda x: x*x

def get_residual_sum_of_squares(model, data, outcome):
    return sum(map(squared, outcome - model.predict(data)))

def get_mean_residual_sum_of_squares(model, data, outcome):
    return sum(map(squared, outcome - model.predict(data)))/len(outcome)

def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    shuffled_data = graphlab.toolkits.cross_validation.shuffle(data, random_seed=1)
    n = len(shuffled_data)
    mse_average = 0
    
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        validation = shuffled_data[start:end]
        train = shuffled_data[0:start].append(shuffled_data[end+1:n])
        model = graphlab.linear_regression.create(train, target = output_name, features = features_list, validation_set = None, l2_penalty=l2_penalty)
        
        mse_average = mse_average + get_mean_residual_sum_of_squares(model, data, data[output_name])
    
    return mse_average/k

def find_best_lambda(folds, input_data, output_name, feature_names):
    penalties = np.logspace(1, 7, num=13)
    lowest_validation_error = float("inf")
    best_lambda = 0
       
    for penalty in penalties:
        mse_average = k_fold_cross_validation(k=folds, l2_penalty=penalty, data=input_data, output_name = output_name, features_list = feature_names)

        if (mse_average < lowest_validation_error):
            lowest_validation_error = mse_average
            best_lambda = penalty

    return best_lambda

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to chen.jenhau@gmail.com and will expire on July 15, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1552157790.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000002  | 2.978388     | 6847177.773672     | 380824.455891 |

| 10        | 11       | 0.000002  | 3.329748     | 6842123.232651     | 364204.576180 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [56]:
model_all['coefficients'].print_rows(num_rows=18)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? <b>bathrooms, sqft_living, sqft_living_sqrt</b>

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [32]:
import numpy as np

l1_penalities = np.logspace(1, 7, num=13)

lowest_validation_error = float("inf")
best_lambda = None

for penalty in l1_penalities:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=penalty)
    
    rss = get_residual_sum_of_squares(model, validation, validation['price'])   
    
    if (rss < lowest_validation_error):
        lowest_validation_error = rss
        best_lambda = penalty

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.857021     | 5740519.699879     | 289427.896727 |

| 6         | 7        | 0.000003  | 0.911259     | 5659149.988446     | 285408.885485 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [35]:
print best_lambda
print l1_penalities

model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=best_lambda)

print "non-zero: ", model['coefficients']['value'].nnz()

10.0
[1.00000000e+01 3.16227766e+01 1.00000000e+02 3.16227766e+02
 1.00000000e+03 3.16227766e+03 1.00000000e+04 3.16227766e+04
 1.00000000e+05 3.16227766e+05 1.00000000e+06 3.16227766e+06
 1.00000000e+07]


Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

...

| 6         | 7        | 0.000003  | 0.746990     | 5655864.782013     | 285354.826809 |

non-zero:  18


+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`? <b>18</b>

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have? <b>10</b>

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [ ]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [42]:
import numpy as np

l1_penalities = np.logspace(8, 10, num=20)

penalty_feature = {}

for penalty in l1_penalities:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=penalty)

    penalty_feature[penalty] = model['coefficients']['value'].nnz()

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000012  | 0.838841     | 7239807.218168     | 388380.843122 |

| 6         | 7        | 0.000012  | 0.890723     | 7187431.218291     | 380705.223778 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [48]:
import operator

sorted_penalty_feature = sorted(penalty_feature.items(), key=operator.itemgetter(1))

print sorted_penalty_feature

[(10000000000.0, 1), (7847599703.514623, 1), (6158482110.6602545, 3), (4832930238.571753, 5), (3792690190.7322536, 6), (2976351441.6313133, 10), (2335721469.0901213, 12), (1832980710.8324375, 13), (1128837891.6846883, 15), (1438449888.2876658, 15), (885866790.4100832, 16), (545559478.1168514, 17), (263665089.87303555, 17), (335981828.6283788, 17), (695192796.1775591, 17), (428133239.8719396, 17), (100000000.0, 18), (206913808.111479, 18), (127427498.57031322, 18), (162377673.91887242, 18)]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [58]:
import math

l1_penalty_min = 2976351441.6313133

print round(l1_penalty_min/1000000.0)*1000000

l1_penalty_max = 3792690190.7322536

print round(l1_penalty_max/1000000.0)*1000000


2976000000.0
3793000000.0


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? <b>l1_penalty_min = 2976351441.6313133
l1_penalty_max = 3792690190.7322536</b>

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [50]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [52]:
import numpy as np

rss_penalty = {}

for penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=penalty)
    rss = get_residual_sum_of_squares(model, validation, validation['price'])

    rss_penalty[rss] = (model['coefficients']['value'].nnz(), penalty)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000003  | 0.774557     | 6700050.222874     | 372113.713871 |

| 6         | 7        | 0.000003  | 0.827586     | 6697293.370032     | 370685.051067 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`? <b>3448968612.163437</b>
2. What features in this model have non-zero coefficients? <b>intercept, bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above</b>

In [61]:
print round(3448968612.163437/1000000)*1000000

3449000000.0


In [55]:
sorted_rss_penalty = sorted(rss_penalty.items(), key=operator.itemgetter(0))

print sorted_rss_penalty

desired_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448968612.163437)

desired_model['coefficients'].print_rows(num_rows=18)

[(966925692362086.5, (10, 2976351441.6313133)), (974019450084557.2, (10, 3019316638.9524155)), (981188367942451.0, (10, 3062281836.2735176)), (989328342459472.0, (10, 3105247033.5946198)), (998783211265885.9, (10, 3148212230.915722)), (1008477167020096.5, (10, 3191177428.236824)), (1018298780553818.6, (10, 3234142625.557926)), (1028247992205976.1, (10, 3277107822.8790283)), (1034616909232829.2, (8, 3320073020.2001305)), (1038554735941039.9, (8, 3363038217.5212326)), (1043237237871704.1, (8, 3406003414.8423347)), (1046937488751713.4, (7, 3448968612.163437)), (1051147625612863.0, (7, 3491933809.484539)), (1055992735342999.4, (7, 3534899006.805641)), (1060799531763290.2, (7, 3577864204.126743)), (1065707689498229.6, (6, 3620829401.447845)), (1069464335425584.1, (6, 3663794598.768947)), (1073504549585594.6, (6, 3706759796.0900493)), (1077632775581412.0, (6, 3749724993.4111514)), (1081867592324111.9, (6, 3792690190.7322536))]


Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

...

| 6         | 7        | 0.000003  | 1.190679     | 6613330.420898     | 360099.555727 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |